# Clasificación de imágenes usando RNNs bidireccionales

* Así como hemos utilizado CNN para clasificación de secuencias, se pueden usar las RNN para clasificación de imágenes.

* Las imágenes son secuencias bidimensionales de píxels.

* En las imágenes, la secuencia es espacial en vez de temporal. Eso nos permite mirar no solo el pasado y el presente, sino también el futuro. Puede parecer ciencia ficción, pero esto solo significa que para un valor de la variable que estoy recorriendo (ejemplo: x o y) no solo tengo acceso a los valores anteriores y actual $x \leq x_0$ sino también a los valores futuros $x > x_0$.

* Como en general tengo acceso a toda la secuencia, tiene sentido utilizar BD-RNN.

<img src="tiempoyespacio.png">

* Puedo interpretar a la imagen como que una dimensión es el tiempo, y otra dimensión son los features.
* La elección de qué dimensión es el tiempo y cual los features es arbitraria.

<img src="tiempoyfeatures.png">

* O también, se pueden implementar ambas opciones:

<img src="dimensiones.png">

- ¿Cómo se implementaría?
- ¿Cómo son las dimensiones en cada punto, si tengo M unidades ocultas, un batch_size de N y la imagen tiene dimensiones WxH?

* Arquitectura entera:

<img src="BD-RNN-clasif.png">

con una capa softmax al final.


In [1]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

X_train, y_train = load_mnist('data/fashion', kind='train')
X_test, y_test = load_mnist('data/fashion', kind='t10k')
X_train=X_train.reshape(-1,28,28)
X_test=X_test.reshape(-1,28,28)

In [ ]:
import os
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Bidirectional, GlobalMaxPooling1D, Lambda, Concatenate, Dense, Conv2D, Reshape, Permute,Flatten
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def get_mnist(limit=None):
  if not os.path.exists('../large_files'):
    print("You must create a folder called large_files adjacent to the class folder first.")
  if not os.path.exists('../large_files/train.csv'):
    print("Looks like you haven't downloaded the data or it's not in the right spot.")
    print("Please get train.csv from https://www.kaggle.com/c/digit-recognizer")
    print("and place it in the large_files folder.")

  print("Reading in and transforming data...")
  df = pd.read_csv('../large_files/train.csv')
  data = df.as_matrix()
  np.random.shuffle(data)
  X = data[:, 1:].reshape(-1, 28, 28) / 255.0 # data is from 0..255
  Y = data[:, 0]
  if limit is not None:
    X, Y = X[:limit], Y[:limit]
  return X, Y


In [ ]:
# get data
X=X_train
Y=y_train
# config
D = 28
M = 100


# input is an image of size 28x28
input_ = Input(shape=(D, D))

# up-down
rnn1 = Bidirectional(LSTM(M, return_sequences=True))
x1 = rnn1(input_) # output is N x D x 2M
x1 = GlobalMaxPooling1D()(x1) # output is N x 2M
#x1=Reshape((1,D,2*M))(x1)
#x1=Permute((1,3,2))(x1)
#x1=Conv2D(1, (1,1), strides=(1, 1), padding='same')(x1)
#x1=Flatten()(x1)


# left-right
rnn2 = Bidirectional(LSTM(M, return_sequences=True))

# custom layer
permutor = Lambda(lambda t: K.permute_dimensions(t, pattern=(0, 2, 1)))

x2 = permutor(input_)
x2 = rnn2(x2) # output is N x D x 2M
x2 = GlobalMaxPooling1D()(x2) # output is N x 2M
#x2=Reshape((1,D,2*M))(x2)
#x2=Permute((1,3,2))(x2)
#x2=Conv2D(1, (1,1), strides=(1, 1), padding='same')(x2)
#x2=Flatten()(x2)
# put them together
concatenator = Concatenate(axis=1)
x = concatenator([x1, x2]) # output is N x 4M

# final dense layer
output = Dense(10, activation='softmax')(x)

model = Model(inputs=input_, outputs=output)

# testing
# o = model.predict(X)
# print("o.shape:", o.shape)

# compile
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
model.summary()

In [ ]:
# train
print('Training model...')
r = model.fit(X, Y, batch_size=64, epochs=40, validation_split=0.1)


# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['acc'], label='acc')
plt.plot(r.history['val_acc'], label='val_acc')
plt.legend()
plt.show()

Training model...
Train on 54000 samples, validate on 6000 samples
Epoch 1/40
54000/54000 [==============================] - 49s 909us/step - loss: 0.5731 - acc: 0.8027 - val_loss: 0.4524 - val_acc: 0.8362
Epoch 2/40
54000/54000 [==============================] - 47s 869us/step - loss: 0.4115 - acc: 0.8484 - val_loss: 0.4006 - val_acc: 0.8532
Epoch 3/40
54000/54000 [==============================] - 47s 869us/step - loss: 0.3822 - acc: 0.8579 - val_loss: 0.3889 - val_acc: 0.8513
Epoch 4/40
54000/54000 [==============================] - 47s 869us/step - loss: 0.3652 - acc: 0.8641 - val_loss: 0.3844 - val_acc: 0.8548
Epoch 5/40
43904/54000 [=======================>......] - ETA: 8s - loss: 0.3509 - acc: 0.8693

In [ ]:
from __future__ import print_function, division
from builtins import range, input

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Bidirectional, Dot, Dense
import numpy as np
import matplotlib.pyplot as plt


T = 8 #Cantidad de Timesteps
D = 2 #Cantidad de entradas por timestep
M = 3 #Cantidad de unidades en la capa oculta


X = np.random.randn(1, D, 2*M)
W = np.random.randn(1, D, 1)
input_ = Input(shape=(D, 2*M))
output = Dot(axes, normalize=False)
print(input)
model = Model(inputs=input_, outputs=x)
o, h1, c1, h2, c2 = model.predict(X)
print("o:", o)
print("o.shape:", o.shape)
print("h1:", h1)
print("c1:", c1)
print("h2:", h2)
print("c2:", c2)

In [32]:
X = np.random.randn(6, D, 2*M)
W = np.random.randn(6, 2*M, D)
y = np.matmul(X,W)

In [33]:
y.shape

(6, 28, 28)